In this part, we deploy the model as a web service. A web service is an image, a Docker image in our case, which encapsulates the scoring logic and the model itself.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
 
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

For deployment, we need three things - a scoring script to show how to use the model, a configuration file to build ACI, and our trained model.

In [ ]:
%%writefile score_unenc.py
import json
import numpy as np
import os
import pickle
import joblib

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes-sklearn-model.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    print('Request data = ', data)
    y_hat = model.predict(data)
    print(y_hat)
    y_classify = lambda k: 1 * (k > 0.5)
    y_hat = y_classify(y_hat)
    print(y_hat)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Create a deployment configuration file specifying number of CPUs and RAM to be used for the ACI container.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "diabetes",  "method" : "sklearn"}, 
                                               description='Predict diabetes with sklearn')

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'diabetes_sklearn')


myenv = Environment.get(workspace=ws, name="diabetes-env", version="1")
inference_config = InferenceConfig(entry_script="score_unenc.py", environment=myenv)

service_name = 'sklearn-diabetes-svc-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.scoring_uri)

In [ ]:
import os
import pandas as pd

data = pd.read_csv("diabetes-test.csv")
print(data.shape)
feature_names = data.iloc[:,0:8].columns
target_name = data.iloc[:1,8:].columns
print(feature_names)
print(target_name)

data_features = data[feature_names]
X_test = data_features.to_numpy(dtype=np.uint8)
data_target = data[target_name]
y_test = data_target.to_numpy(dtype=np.uint8).reshape(-1)

print(X_test)
print(y_test)


In [ ]:
import json
test = json.dumps({"data": X_test.tolist()})
test = bytes(test, encoding='utf8')
y_hat = service.run(input_data=test)

print(y_hat)

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mx = confusion_matrix(y_test, y_hat)
print(conf_mx)
print('Overall accuracy:', np.average(y_hat == y_test))

In [ ]:
# normalize the diagonal cells so that they don't overpower the rest of the cells when visualized
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
np.fill_diagonal(norm_conf_mx, 0)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
cax = ax.matshow(norm_conf_mx, cmap=plt.cm.bone)
ticks = np.arange(0, 2, 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(ticks)
ax.set_yticklabels(ticks)
fig.colorbar(cax)
plt.ylabel('true labels', fontsize=14)
plt.xlabel('predicted values', fontsize=14)
plt.savefig('conf.png')
plt.show()

In [ ]:
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test)-1)
print('Random index = ', random_index)
input_data = "{\"data\": [" + str(list(X_test[random_index])) + "]}"
print('input 1 : ', input_data)

headers = {'Content-Type':'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("label:", y_test[random_index])
print("prediction:", resp.text)

In [ ]:
service.delete()